# "Bladder_AUA" feature selection - ExtraTrees

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as st
import sklearn.ensemble as en
import sklearn.feature_selection as fs
import sklearn.metrics as mt
import sklearn.model_selection as ms
import sklearn.preprocessing as pp
import sklearn.tree as tr
import time as tm
import xgboost as xgb
from sklearn.pipeline import Pipeline

In [ ]:
RANDOM_STATE=4+8+15+16+23+42
NUM_FEATURES_TO_SELECT=10
CROSS_VALIDATION_FOLDS=10#3,5,10
CROSS_VALIDATION_REPEATS=100#10, 100, 1000

In [ ]:
# Cargar datos
X_raw = pd.read_csv("/content/drive/MyDrive/Documentos/7. educación/uniovi/master/tfm/data/Bladder_AUA_features.csv")
y_raw = pd.read_csv("/content/drive/MyDrive/Documentos/7. educación/uniovi/master/tfm/data/Bladder_AUA_labels.csv")
y = y_raw.iloc[:, 1:].values.ravel() # Las etiquetas están en la segunda columna

In [ ]:
# Guardar los nombres de las características
original_feature_names = X_raw.columns.tolist()

# Preprocesamiento
pipe = Pipeline([
  ("feature_elimination", fs.VarianceThreshold()),
  ("scaler", pp.StandardScaler())
])
X_preprocessed = pipe.fit_transform(X_raw)
preprocessed_feature_names = pipe.get_feature_names_out(input_features=original_feature_names)
X = pd.DataFrame(X_preprocessed, columns=preprocessed_feature_names)

In [ ]:
# Definir la estrategia de validación cruzada
cv_strategy = ms.RepeatedStratifiedKFold(n_splits = CROSS_VALIDATION_FOLDS,
                               n_repeats = CROSS_VALIDATION_REPEATS,
                               random_state=RANDOM_STATE)

In [ ]:
# Tomar el tiempo inicial
start_time = tm.time()

# Entrenar el modelo y añadir las características a la lista
feature_importances_list = []
for train_index, test_index in cv_strategy.split(X, y):
  model = en.ExtraTreesClassifier(random_state=RANDOM_STATE)
  model.fit(X.iloc[train_index], y[train_index])
  feature_importances_list.append(model.feature_importances_)

# Calcular la media
averaged_feature_importances = np.mean(np.array(feature_importances_list), axis=0)

# Seleccionar las mejores características
selector = fs.SelectFromModel(estimator=en.ExtraTreesClassifier(random_state=RANDOM_STATE),
                              threshold=-np.inf,
                              max_features=NUM_FEATURES_TO_SELECT,
                              importance_getter=lambda x: averaged_feature_importances)
selector.fit(X, y)
X_selected = selector.transform(X)
selected_feature_names = selector.get_feature_names_out(input_features=X.columns)

# Tomar el tiempo final
end_time = tm.time()

print(f"*** Tiempo: {(end_time - start_time):.2f} segundos")
print(f"*** Selector: {selector.__class__.__name__}")
print(f"*** Estimador: {selector.estimator_.__class__.__name__}")
print(
  f"*** Características totales/seleccionadas: "
  f"{len(original_feature_names)}/{X_selected.shape[1]}"
)
print(f"")
for feature_name in selected_feature_names:
    print(f"  - {feature_name}")
print("-" * 60)